In [1]:
import findspark
findspark.init('c:/spark')
from pyspark import SparkConf, SparkContext
import collections
import re
from pyspark.sql import SparkSession
from pyspark.sql import Row
import pandas as pd
import collections
from pyspark.ml.linalg import Vectors
import glob

In [2]:
spark = SparkSession.builder.config("spark.sql.warehouse.dir", "C:/temp").appName("SparkSQL").getOrCreate()

In [3]:
allFiles = glob.glob("./newdata3.parquet/*.parquet")


In [4]:
train=spark.read.parquet(*allFiles)

In [5]:
train.count

<bound method DataFrame.count of DataFrame[log_duration: double, log_trip_duration: double, pickup_longitude: double, pickup_latitude: double, dropoff_longitude: double, dropoff_latitude: double, great_circle_distance: double, distance: double, snow: bigint, holiday: bigint, vendor_id: bigint, pickup_hour_0: bigint, pickup_hour_1: bigint, pickup_hour_2: bigint, pickup_hour_3: bigint, pickup_hour_4: bigint, pickup_hour_5: bigint, pickup_hour_6: bigint, pickup_hour_7: bigint, pickup_hour_8: bigint, pickup_hour_9: bigint, pickup_hour_10: bigint, pickup_hour_11: bigint, pickup_hour_12: bigint, pickup_hour_13: bigint, pickup_hour_14: bigint, pickup_hour_15: bigint, pickup_hour_16: bigint, pickup_hour_17: bigint, pickup_hour_18: bigint, pickup_hour_19: bigint, pickup_hour_20: bigint, pickup_hour_21: bigint, pickup_hour_22: bigint, pickup_hour_23: bigint, pickup_weekday_0: bigint, pickup_weekday_1: bigint, pickup_weekday_2: bigint, pickup_weekday_3: bigint, pickup_weekday_4: bigint, pickup_we

In [6]:
topcoord=pd.read_csv("coordinate_more_than_1000.csv")
coordinates_name=topcoord['coordinates'].tolist()
coordinates_name2=topcoord['valname'].tolist()
feature=['log_duration', 'log_trip_duration', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'great_circle_distance','distance',
       'snow', 'holiday', 'vendor_id', 'pickup_hour_0',
       'pickup_hour_1', 'pickup_hour_2', 'pickup_hour_3', 'pickup_hour_4',
       'pickup_hour_5', 'pickup_hour_6', 'pickup_hour_7', 'pickup_hour_8',
       'pickup_hour_9', 'pickup_hour_10', 'pickup_hour_11',
       'pickup_hour_12', 'pickup_hour_13', 'pickup_hour_14',
       'pickup_hour_15', 'pickup_hour_16', 'pickup_hour_17',
       'pickup_hour_18', 'pickup_hour_19', 'pickup_hour_20',
       'pickup_hour_21', 'pickup_hour_22', 'pickup_hour_23',
       'pickup_weekday_0', 'pickup_weekday_1', 'pickup_weekday_2',
       'pickup_weekday_3', 'pickup_weekday_4', 'pickup_weekday_5',
       'pickup_weekday_6', 'pickup_month_1', 'pickup_month_2',
       'pickup_month_3', 'pickup_month_4', 'pickup_month_5',
       'pickup_month_6', 'passenger_count_0', 'passenger_count_1',
       'passenger_count_2', 'passenger_count_3', 'passenger_count_4',
       'passenger_count_5', 'passenger_count_6', 'passenger_count_7']
colNames=feature+coordinates_name2

In [7]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import OneHotEncoder

In [8]:
feature_col=colNames[0:1]+colNames[2:]
vecAssembler = VectorAssembler(inputCols=feature_col, outputCol="features")


In [9]:
(trainingData, testData) = train.randomSplit([0.8, 0.2],seed=42)

In [10]:

gbt = GBTRegressor(featuresCol="features",labelCol="log_trip_duration",maxIter=10,maxDepth=10, seed=42)

# Chain indexer and GBT in a Pipeline
pipeline = Pipeline(stages=[vecAssembler,gbt])

# Train model.  This also runs the indexer.
model = pipeline.fit(trainingData)

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "C:\Users\TANGP05\AppData\Local\Continuum\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-c7441f334ba7>", line 8, in <module>
    model = pipeline.fit(trainingData)
  File "c:/spark\python\pyspark\ml\base.py", line 64, in fit
    return self._fit(dataset)
  File "c:/spark\python\pyspark\ml\pipeline.py", line 108, in _fit
    model = stage.fit(dataset)
  File "c:/spark\python\pyspark\ml\base.py", line 64, in fit
    return self._fit(dataset)
  File "c:/spark\python\pyspark\ml\wrapper.py", line 265, in _fit
    java_model = self._fit_java(dataset)
  File "c:/spark\python\pyspark\ml\wrapper.py", line 262, in _fit_java
    return self._java_obj.fit(dataset._jdf)
  File "c:\spark\python\lib\py4j-0.10.4-src.zip\py4j\java_gateway.py", line 1131, in __call__
    answer = self.gateway_client.se

KeyboardInterrupt: 